# Preparing Data For Model Training and Testing

After running **conf-rank-scraper.py**, **final-four-champ-scraper.py**, **sr-scraper.py** and **tourn-seed-scraper1.py**, there will be multiple .csv files in the present working directory.  Running this notebook will join the .csv files into a single file, **d1-basketball-data-93-23.csv**.  

## Aggregating .csv files into a single dataframe

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:

ff_champ_df = pd.read_csv("final-four-champ.csv")
final_df = pd.DataFrame()

# looping through each year that there are .csv files for
for season in range(1993, 2024):
    
    # there was no post season in 2020
    if season==2020:
        pass
    
    else:
        '''
        for every season, the relevant season-stats, conf-rankings, and tourn-seed .csv files are stored
        into variables as DataFrames
        '''
        season_df = pd.read_csv("season-stats-{}.csv".format(str(season)))
        conf_rank_df = pd.read_csv("conf-rankings-{}.csv".format(str(season)))
        tourn_seed_df = pd.read_csv("tourn-seeds-{}.csv".format(str(season)), names=["region_seed","School","region"])

        '''
        not a real rank, it is an alphabetical index and this does not affect a team's ability to 
        qualify for the NCAA tournament
        '''
        season_df.drop("Rk", axis=1, inplace=True)

        '''
        Multiple string representations were used for the same team.   The following code changes
        different string representations to the same one so the different dataframes may be 
        later merged into a larger dataframe for the season/ year. 
        '''
        season_df["School"] = list(map(lambda x: x.replace("NCAA","").strip(),season_df["School"]))
        season_df["School"] = list(map(lambda x: "Nevada Las Vegas" if x=="Nevada-Las Vegas" else x, season_df["School"]))
        season_df["School"] = list(map(lambda x: "Illinois Chicago" if x=="Illinois-Chicago" else x, season_df["School"]))

        # remove blank column
        conf_rank_df.drop("Unnamed: 0", inplace=True, axis=1)
        # redundant data is dropped, only new/ relevant data remains in the dataframe now
        conf_rank_df.drop(conf_rank_df.iloc[:, 3:23], inplace=True, axis=1)
        conf_rank_df.columns = ["Conf_Rk", "School", "Conf"]
        
        # conferences were represented with different strings, this makes them the same
        conf_rank_df["Conf"] = list(map(lambda x: re.sub(r"\s\(\w*\s*\w*\s*\d*\)", r"", str(x)), conf_rank_df["Conf"]))

        season_df = season_df.merge(conf_rank_df, on="School", how="left")

        # an un-needed column
        tourn_seed_df.drop("region", axis=1, inplace=True)

        # more changing the names of schools so they are the same across dataframes
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UNC","North Carolina"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UMass","Massachusetts"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("Penn","Pennsylvania"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("LSU","Louisiana State"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("BYU","Brigham Young"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("SMU","Saint Mary's (CA)"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("Pitt","Pittsburgh"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UCF","Central Florida"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("VCU","Virginia Commonwealth"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("Ole Miss","Mississippi"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("USC","South Carolina"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UNLV","Nevada Las Vegas"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UIC","Illinois Chicago"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("ETSU","East Tennessee State"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UCSB","UC Santa Barbara"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("LIU","Long Island University"), tourn_seed_df["School"]))
        tourn_seed_df["School"] = list(map(lambda x: x.replace("UConn","Connecticut"), tourn_seed_df["School"]))

        season_df = season_df.merge(tourn_seed_df, on="School", how="left")

        # creating the target feature for the machine learning models
        season_df["qual_for_ncaa"] = list(map(lambda x: pd.notna(x), season_df["region_seed"]))

        ff_champ_df["champ"] = list(map(lambda x: "North Carolina" if x=="UNC" else x, ff_champ_df["champ"]))
        ff_champ_df["champ"] = list(map(lambda x: "Connecticut" if x=="UConn" else x, ff_champ_df["champ"]))
        ff_champ_df["final_four"] = list(map(lambda x: x.replace("UNC","North Carolina"), ff_champ_df["final_four"]))
        ff_champ_df["final_four"] = list(map(lambda x: x.replace("UConn","Connecticut"), ff_champ_df["final_four"]))
        ff_champ_df["final_four"] = list(map(lambda x: x.replace("LSU","Louisiana State"), ff_champ_df["final_four"]))
        ff_champ_df["final_four"] = list(map(lambda x: x.replace("VCU","Virginia Commonwealth"), ff_champ_df["final_four"]))

        # creating more target features for future machine learning models
        season_df["final_four"] = list(map(lambda x: x in ff_champ_df.iloc[season-1993,1].split("-"), season_df["School"]))
        season_df["ncaa_champ"] = list(map(lambda x: x== ff_champ_df.iloc[season-1993,2], season_df["School"]))

        season_df["season"] = str(season)

        final_df = pd.concat([final_df, season_df], ignore_index=True)
   
final_df.head()    


,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8,W.1,L.1,...,TOV%,ORB%,FT/FGA,Conf_Rk,Conf,region_seed,qual_for_ncaa,final_four,ncaa_champ,season
0,Air Force,28,9,19,0.321,-7.45,2.05,NaN,3.0,15.0,...,17.4,NaN,0.263,10.0,WAC,NaN,False,False,False,1993
1,Akron,26,8,18,0.308,-10.69,-5.06,NaN,3.0,15.0,...,18.4,NaN,0.241,10.0,MAC,NaN,False,False,False,1993
2,Alabama,29,16,13,0.552,9.66,7.83,NaN,7.0,9.0,...,19.4,NaN,0.271,10.0,SEC,NaN,False,False,False,1993
3,Alabama State,27,14,13,0.519,-8.49,-9.70,NaN,9.0,5.0,...,20.7,NaN,0.305,3.0,SWAC,NaN,False,False,False,1993
4,Alcorn State,27,7,20,0.259,-11.04,-4.58,NaN,5.0,9.0,...,19.7,NaN,0.188,7.0,SWAC,NaN,False,False,False,1993


## Dropping features

The respective columns of features that have NaN values present in less than 98% of the rows were dropped.  

In [3]:
for col in final_df.columns:
    print(col+": "+str(sum(pd.notna(final_df[col]))/final_df.shape[0]))


School: 1.0
G: 1.0
W: 1.0
L: 1.0
W-L%: 0.9991997599279784
SRS: 0.9920976292887866
SOS: 0.9920976292887866
Unnamed: 8: 0.0
W.1: 0.9852955886766029
L.1: 0.9852955886766029
Unnamed: 11: 0.0
W.2: 0.9981994598379513
L.2: 0.9981994598379513
Unnamed: 14: 0.0
W.3: 0.9985995798739622
L.3: 0.9985995798739622
Unnamed: 17: 0.0
Tm.: 0.998999699909973
Opp.: 0.9991997599279784
Unnamed: 20: 0.0
Pace: 0.5940782234670401
ORtg: 0.6146844053215965
FTr: 0.9988996699009703
3PAr: 0.9988996699009703
TS%: 0.9988996699009703
TRB%: 0.9985995798739622
AST%: 0.9988996699009703
STL%: 0.6146844053215965
BLK%: 0.9984995498649595
eFG%: 0.9988996699009703
TOV%: 0.995998799639892
ORB%: 0.6170851255376613
FT/FGA: 0.9988996699009703
Conf_Rk: 0.9895968790637191
Conf: 0.9895968790637191
region_seed: 0.1871561468440532
qual_for_ncaa: 1.0
final_four: 1.0
ncaa_champ: 1.0
season: 1.0


In [4]:
final_df.drop(["Unnamed: 8", "Unnamed: 11", "Unnamed: 14", 
               "Unnamed: 17", "Unnamed: 20", "Pace", "ORtg", 
               "STL%", "ORB%"], inplace=True, axis=1)
final_df


,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,eFG%,TOV%,FT/FGA,Conf_Rk,Conf,region_seed,qual_for_ncaa,final_four,ncaa_champ,season
0,Air Force,28,9,19,0.321,-7.45,2.05,3.0,15.0,3.0,...,0.458,17.4,0.263,10.0,WAC,NaN,False,False,False,1993
1,Akron,26,8,18,0.308,-10.69,-5.06,3.0,15.0,4.0,...,0.488,18.4,0.241,10.0,MAC,NaN,False,False,False,1993
2,Alabama,29,16,13,0.552,9.66,7.83,7.0,9.0,11.0,...,0.513,19.4,0.271,10.0,SEC,NaN,False,False,False,1993
3,Alabama State,27,14,13,0.519,-8.49,-9.70,9.0,5.0,7.0,...,0.518,20.7,0.305,3.0,SWAC,NaN,False,False,False,1993
4,Alcorn State,27,7,20,0.259,-11.04,-4.58,5.0,9.0,3.0,...,0.487,19.7,0.188,7.0,SWAC,NaN,False,False,False,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,Wright State,33,18,15,0.545,-2.33,-5.87,10.0,10.0,9.0,...,0.546,15.3,0.178,6.0,Horizon,NaN,False,False,False,2023
9993,Wyoming,31,9,22,0.290,2.01,6.11,4.0,14.0,7.0,...,0.520,16.0,0.236,11.0,MWC,NaN,False,False,False,2023
9994,Xavier,37,27,10,0.730,16.03,9.16,15.0,5.0,15.0,...,0.552,15.0,0.223,2.0,Big East,3.0,True,False,False,2023
9995,Yale,30,21,9,0.700,7.81,-1.62,10.0,4.0,10.0,...,0.540,14.3,0.208,2.0,Ivy,NaN,False,False,False,2023


## Imputing values

Teams that did not qualify for the NCAA tournament were given a "region_seed" of -1.

In [5]:
final_df["region_seed"] = list(map(lambda x: -1 if pd.isna(x) else x, final_df["region_seed"]))
final_df

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,eFG%,TOV%,FT/FGA,Conf_Rk,Conf,region_seed,qual_for_ncaa,final_four,ncaa_champ,season
0,Air Force,28,9,19,0.321,-7.45,2.05,3.0,15.0,3.0,...,0.458,17.4,0.263,10.0,WAC,-1,False,False,False,1993
1,Akron,26,8,18,0.308,-10.69,-5.06,3.0,15.0,4.0,...,0.488,18.4,0.241,10.0,MAC,-1,False,False,False,1993
2,Alabama,29,16,13,0.552,9.66,7.83,7.0,9.0,11.0,...,0.513,19.4,0.271,10.0,SEC,-1,False,False,False,1993
3,Alabama State,27,14,13,0.519,-8.49,-9.70,9.0,5.0,7.0,...,0.518,20.7,0.305,3.0,SWAC,-1,False,False,False,1993
4,Alcorn State,27,7,20,0.259,-11.04,-4.58,5.0,9.0,3.0,...,0.487,19.7,0.188,7.0,SWAC,-1,False,False,False,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,Wright State,33,18,15,0.545,-2.33,-5.87,10.0,10.0,9.0,...,0.546,15.3,0.178,6.0,Horizon,-1,False,False,False,2023
9993,Wyoming,31,9,22,0.290,2.01,6.11,4.0,14.0,7.0,...,0.520,16.0,0.236,11.0,MWC,-1,False,False,False,2023
9994,Xavier,37,27,10,0.730,16.03,9.16,15.0,5.0,15.0,...,0.552,15.0,0.223,2.0,Big East,3.0,True,False,False,2023
9995,Yale,30,21,9,0.700,7.81,-1.62,10.0,4.0,10.0,...,0.540,14.3,0.208,2.0,Ivy,-1,False,False,False,2023


## Drop all rows with NaN values

After dropping all rows with NaN values, we retain nearly 97% of the rows in our aggregated dataframe

In [6]:
final_df.dropna(axis=0, inplace=True)
final_df

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,eFG%,TOV%,FT/FGA,Conf_Rk,Conf,region_seed,qual_for_ncaa,final_four,ncaa_champ,season
0,Air Force,28,9,19,0.321,-7.45,2.05,3.0,15.0,3.0,...,0.458,17.4,0.263,10.0,WAC,-1,False,False,False,1993
1,Akron,26,8,18,0.308,-10.69,-5.06,3.0,15.0,4.0,...,0.488,18.4,0.241,10.0,MAC,-1,False,False,False,1993
2,Alabama,29,16,13,0.552,9.66,7.83,7.0,9.0,11.0,...,0.513,19.4,0.271,10.0,SEC,-1,False,False,False,1993
3,Alabama State,27,14,13,0.519,-8.49,-9.70,9.0,5.0,7.0,...,0.518,20.7,0.305,3.0,SWAC,-1,False,False,False,1993
4,Alcorn State,27,7,20,0.259,-11.04,-4.58,5.0,9.0,3.0,...,0.487,19.7,0.188,7.0,SWAC,-1,False,False,False,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,Wright State,33,18,15,0.545,-2.33,-5.87,10.0,10.0,9.0,...,0.546,15.3,0.178,6.0,Horizon,-1,False,False,False,2023
9993,Wyoming,31,9,22,0.290,2.01,6.11,4.0,14.0,7.0,...,0.520,16.0,0.236,11.0,MWC,-1,False,False,False,2023
9994,Xavier,37,27,10,0.730,16.03,9.16,15.0,5.0,15.0,...,0.552,15.0,0.223,2.0,Big East,3.0,True,False,False,2023
9995,Yale,30,21,9,0.700,7.81,-1.62,10.0,4.0,10.0,...,0.540,14.3,0.208,2.0,Ivy,-1,False,False,False,2023


In [7]:
9695/9997

0.9697909372811844

## Saving the final dataframe to a .csv file

In [8]:
final_df.to_csv("d1-basketball-data-93-23.csv")